Initialising the database in memory

In [26]:
import sqlite3
import pandas as pd

db = "bbfa-db.db"

# Connect to database
conn = sqlite3.connect(db)

# Open the cursor
cursor = conn.cursor()

# Open the external sql file.
file = open("create_table.sql", 'r')
    
# Read out the sql script text in the file.
sql_script_string = file.read()
    
# Close the sql file object.
file.close()

# Run the sql script string
cursor.executescript(sql_script_string)

# Commit the changes
conn.commit()

# Close connection
conn.close()


OperationalError: table Customer already exists

In [27]:
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c) 

def show_tables():
    q = '''
        SELECT
            name
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)

def get_table_row_count(tablename):
    q = '''
        SELECT
            COUNT(1)
        FROM %s;
        ''' % tablename
    return run_query(q)["COUNT(1)"][0]

tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables["name"]]
print(tables)


             name  row_count
0        Customer       1000
1          Visits       1000
2           makes       1000
3   shoppingCarts       1000
4             has       1000
5   purchasedCart        600
6   abandonedCart        400
7        Products         10
8        contains       1000
9        Category          5
10      BelongsTo         10


In [29]:

import datapane as dp 

top_5_selling_products = '''
SELECT t.productID, SUM(t.quantity)
FROM
(SELECT c.cartID,c.productID, c.quantity
FROM contains c
WHERE c.cartID in (SELECT cartID from purchasedcart)) t
group BY t.productID
order by SUM(t.quantity) desc
limit 5;

'''

top_5_df = run_query(top_5_selling_products)

dp.Report(
    dp.DataTable(top_5_df)
).upload(name="My first report")

ImportError: pyarrow>=1.0.0 is required for PyArrow backed StringArray.